In [1]:
%load_ext autoreload
%autoreload 2

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import pickle
from tqdm import trange
try:
  from helpers import get_model_size
except ModuleNotFoundError:
  import sys
  sys.path.append("../")
  from helpers import get_model_size

In [4]:
# we actually just need it to download cifar dataset
torchvision.datasets.CIFAR10(train=True, download=True, root='../data/', transform=transforms.ToTensor())
torchvision.datasets.CIFAR10(train=False, download=True, root='../data/', transform=transforms.ToTensor())

100.0%


Extracting ../data/cifar-10-python.tar.gz to ../data/
Files already downloaded and verified


In [2]:
class CiFaData(Dataset):
  def __init__(self, stage="train", transform=None, device="cpu"):
    self.device = device
    self.base_folder = "cifar-10-batches-py"
    self.transform = transform
    if stage == "train":
      batch_collection = [f"data_batch_{i}" for i in range(1, 5)]
    elif stage == "val":
      batch_collection = ["data_batch_5"]
    elif stage == "test":
      batch_collection = ["test_batch"]
    else:
      raise ValueError("Invalid stage, choose from train, val, test.")
    self.x_data = []
    self.y_data = []
    for batch in batch_collection:
      with open(f"../data/cifar-10-batches-py/{batch}", "rb") as f:
        data = pickle.load(f, encoding="latin1") 
        self.x_data.extend(data["data"])
        self.y_data.extend(data["labels"])
    self.y_data = torch.tensor(self.y_data)
    self.x_data = np.vstack(self.x_data).reshape(-1, 3, 32, 32) # from list to np stack; results in (N, 3, 32, 32)
    self.x_data = self.x_data.transpose((0, 2, 3, 1)) # into (N, H, W, C)
  def __len__(self):
    return len(self.y_data)
  def __getitem__(self, idx):
    if self.transform:
      return self.transform(self.x_data[idx]), self.y_data[idx]
    return transforms.ToTensor()(self.x_data[idx]).to(self.device), self.y_data[idx].to(self.device)

In [3]:
tf = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
# tf = transforms.Normalize(0.5, 0.5)

# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'

train_ds = CiFaData(stage="train", device=device)
cuda_ds = CiFaData(stage="train", device='cuda')
cpu_ds = CiFaData(stage="train", device='cpu')
val_ds = CiFaData(stage="val", device=device)
test_ds = CiFaData(stage="test", device=device)

In [6]:
# pinning memory, takes cpu data and pins it to the gpu. meaning if I already 
cpu_loader = DataLoader(cpu_ds, batch_size=1024, shuffle=True)
cuda_loader = DataLoader(cuda_ds, batch_size=128, shuffle=True) 
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)

In [12]:
# using the Sequential container
class LittleConvSeq(nn.Module):
  def __init__(self):
    super().__init__()
    self.net = nn.Sequential(
    nn.Conv2d(3,6,5), # out: (B, 6, 28, 28)
    nn.ReLU(),
    nn.MaxPool2d(2,2), # (B, 6, 14, 14)
    nn.Flatten(1),
    nn.Linear(6 *14*14, 10) # (B, 10)
    )
  def forward(self, x):
    # x = F.relu(self.conv1(x))
    # x = self.pool(x)
    # x = self.fc1(torch.flatten(x,1))
    return self.net(x)

In [18]:
# without using the Sequential container
class LittleConv(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3,6,5) # out: (B, 6, 28, 28)
    self.pool = nn.MaxPool2d(2,2) # (B, 6, 14, 14)
    self.fc1 = nn.Linear(6 *14*14, 10) # (B, 10)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = self.fc1(torch.flatten(x,1))
    return x

In [19]:
# with cuda: bs32 - 2.44s; bs64 - 2.47; bs128 - 2.31s

epochs = 20

littleconv = LittleConv()
optimimizer = optim.SGD(lr=0.001, params=littleconv.parameters(), momentum=0.9)
criterion = nn.CrossEntropyLoss()
littleconv.to('cuda')

for i in (t:= trange(epochs)):
  for x, y in cuda_loader:
    optimimizer.zero_grad()
    pred = littleconv(x)  
    loss = criterion(pred, y)
    loss.backward()
    optimimizer.step()

  t.set_description(f"epoch {i+1}: loss: {loss.item():.4f}")
f"that took {t.format_interval(t.format_dict['elapsed'])} minutes"

epoch 20: loss: 1.5168: 100%|██████████| 20/20 [00:46<00:00,  2.30s/it]


'that took 00:46 minutes'

In [8]:
# cpu: bs 32 - 1.76s; bs64 - 1.42; bs128 - 1.28s

epochs = 20

littleconv = LittleConv()
optimimizer = optim.SGD(lr=0.001, params=littleconv.parameters(), momentum=0.9)
criterion = nn.CrossEntropyLoss()
littleconv.to('cpu')

for i in (t:= trange(epochs)):
  for x, y in cpu_loader:
    optimimizer.zero_grad()
    pred = littleconv(x)  
    loss = criterion(pred, y)
    loss.backward()
    optimimizer.step()

  t.set_description(f"epoch {i+1}: loss: {loss.item():.4f}")
f"that took {t.format_interval(t.format_dict['elapsed'])} minutes"

epoch 20: loss: 1.9749: 100%|██████████| 20/20 [00:31<00:00,  1.59s/it]


'that took 00:31 minutes'

In [55]:
epochs = 20

net.to('cpu')

# littleconv = LittleConv()
optimimizer = optim.SGD(lr=0.001, params=net.parameters(), momentum=0.9)
criterion = nn.CrossEntropyLoss()

for i in (t:= trange(epochs)):
  for x, y in cpu_loader:
    optimimizer.zero_grad()
    pred = net(x)  
    loss = criterion(pred, y)
    loss.backward()
    optimimizer.step()

  t.set_description(f"epoch {i+1}: loss: {loss.item():.4f}")

epoch 20: loss: 2.3024: 100%|██████████| 20/20 [00:37<00:00,  1.85s/it]


In [62]:
epochs = 20

net.to('cuda')

# littleconv = LittleConv()
optimimizer = optim.SGD(lr=0.001, params=net.parameters(), momentum=0.9)
criterion = nn.CrossEntropyLoss()

for i in (t:= trange(epochs)):
  for x, y in cuda_loader:
    optimimizer.zero_grad()
    pred = net(x)  
    loss = criterion(pred, y)
    loss.backward()
    optimimizer.step()

  t.set_description(f"epoch {i+1}: loss: {loss.item():.4f}")

epoch 20: loss: 1.9571: 100%|██████████| 20/20 [00:40<00:00,  2.05s/it]


ResNet 18

In [11]:
# example net

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

In [ ]:
class Res18Blocks(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 3, 64)
    self.conv2 = nn.Conv2d(3, 3, 64)
    

In [20]:
class ResNet(nn.Module):
  def __init__(self, res_blocks):
    super().__init__()
    self.conv0 = nn.Conv2d(7, 7, 64, stride=2)
    self.mpool = nn.MaxPool2d(3, 3, stride=2)
    self.avgpool = nn.AvgPool2d()
    self.conv1 = res_blocks[0]
    self.con2 = res_blocks[1]
    self.conv3 = res_blocks[2]
    self.conv4 = res_blocks[3]
    self.conv5 = res_blocks[4]
    self.fc = nn.Linear(10) # number of classes

  
  def forward(self, x):
    x = self.conv0(x)
    x = self.mpool(x)
    x += self.conv1(x)
    x += self.con2(x)
    x += self.conv3(x)
    x += self.conv4(x)
    x += self.conv5(x)
    x = self.avgpool(x)
    x = self.fc(x)

    return x